In [24]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")

#print(df.head(10))
print(df.shape)
print(df.info())

(891, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


## 🔱주요 컬럼:

PassengerId: 승객 ID

Survived: 생존 여부 (0 = 사망, 1 = 생존)

Pclass: 객실 등급 (1, 2, 3)

Name: 이름 (예: Braund, Mr. Owen Harris)

Sex: 성별 (male, female)

Age: 나이 (결측값 다수 포함)

SibSp: 함께 탑승한 형제자매/배우자 수

Parch: 함께 탑승한 부모/자녀 수

Ticket: 티켓 번호

Fare: 운임

Cabin: 선실 번호 (예: C85, 결측값 매우 많음)

Embarked: 탑승 항구 (S, C, Q)

💻 제1유형 연습 문제

문제 1. (결측값 채우기 - 고급) Age 컬럼의 결측값(NaN)이 많습니다. 이 결측값을 **Pclass (객실 등급) 별 Age의 중앙값(median)**으로 채워 넣으려고 합니다. (예: 1등석(Pclass=1) 승객의 Age 결측값은 1등석 승객 전체의 Age 중앙값으로 채움) 결측값을 모두 채운 후, Age 컬럼의 전체 평균값을 구하시오. (단, 소수점 셋째 자리에서 반올림하여 둘째 자리까지 계산)

문제 2. (문자열 추출 - Regex) Name 컬럼에는 승객의 호칭(Title)이 포함되어 있습니다. (예: Mr., Mrs., Miss., Master. 등) Name 컬럼에서 **정규표현식(Regex)**을 사용하여 이 호칭([A-Za-z]+\.)을 추출하여 'Title'이라는 새 컬럼을 만드시오. 그 후, 'Title'별 인원수를 집계할 때, 10명 이상의 인원을 가진 'Title' 그룹들의 Age 평균을 구하시오. (단, 소수점 셋째 자리에서 반올림하여 둘째 자리까지 계산)

문제 3. (결측값 + 문자열 처리 - 복합) Cabin 컬럼은 결측값이 매우 많습니다.

Cabin 컬럼의 결측값은 'Unknown'으로 채웁니다.

Cabin 컬럼이 결측값이 아닌 경우, 값의 **첫 번째 알파벳(Deck)**만 추출합니다. (예: C85 -> C, G6 -> G)

Unknown으로 채워진 데이터는 Deck를 'U'로 변경합니다. (힌트: fillna('U') 후 .str[0])

이렇게 생성된 Deck 컬럼(A, B, C... U)별 Survived (생존율) 평균을 계산할 때, 생존율이 가장 높았던 Deck의 이름을 출력하시오.

In [25]:
# 1번 문제(transform)
# df.groupby('<group_col>')['<calc_col>'].transform('<function>')

age_fill_values = df.groupby('Pclass')['Age'].transform('median')
df['Age'] = df['Age'].fillna(age_fill_values)

result = df['Age'].mean()
print(round(result, 2))


29.07


In [26]:
# 1번 문제(loc)
# df.loc[<row_condition>, <column_name>] = new_value

medians = df.groupby('Pclass')['Age'].median()
# medians[1] = 1등석 중앙값
# medians[2] = 2등석 중앙값
# medians[3] = 3등석 중앙값

df.loc[(df['Pclass'] == 1) & (df['Age'].isna()), 'Age'] = medians[1]
df.loc[(df['Pclass'] == 2) & (df['Age'].isna()), 'Age'] = medians[2]
df.loc[(df['Pclass'] == 3) & (df['Age'].isna()), 'Age'] = medians[3]

result = df['Age'].mean()
print(round(result, 2))

print(medians[1])

29.07
37.0


🚀 요약: 그래서 언제 뭘 쓸까?
loc (수술): "딱 이 칸, 저 칸만 골라서" 값을 바꾸고 싶을 때.

df.loc[<조건>, <바꿀컬럼>] = <새값>

transform (공정): "그룹별 계산 값을" "원본 모든 행에" 매핑해서 사용하고 싶을 때.

df.groupby(...).transform(...) (주로 fillna와 찰떡궁합입니다.)

In [30]:
# 2번 문제
df['Title'] = df['Name'].str.extract('([A-Za-z]+)\.')
title_counts = df['Title'].value_counts()

target_titles = title_counts[title_counts >= 10].index
target_df = df[df['Title'].isin(target_titles)]
result = target_df['Age'].mean()
print(round(result, 2))

28.66
